# Importação das bibliotecas

Sendo executado no ambiente virtual: teste.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

# Abertura do arquivo

O arquivo aberto é diário referente ao ano de 2021.

In [2]:
ds = xr.open_dataset('./dados/dados_ambientais.nc')

In [3]:
ds

<xarray.Dataset>
Dimensions:  (time: 365, lon: 1, lat: 1)
Coordinates:
  * time     (time) datetime64[ns] 2021-01-01 2021-01-02 ... 2021-12-31
  * lon      (lon) float64 0.0
  * lat      (lat) float64 0.0
Data variables:
    prec     (time, lat, lon) float32 ...
    nf       (time, lat, lon) float32 ...
    rh2m     (time, lat, lon) float32 ...
    tmax     (time, lat, lon) float64 ...
    tmed     (time, lat, lon) float64 ...
    tmin     (time, lat, lon) float64 ...
Attributes:
    CDI:                  Climate Data Interface version 2.0.5 (https://mpime...
    Conventions:          COARDS
    History:              created by wgrib2
    GRIB2_grid_template:  0
    history:              Tue May 03 10:31:16 2022: cdo merge chuva.2021.nc n...
    CDO:                  Climate Data Operators version 2.0.5 (https://mpime...

In [4]:
type(ds)

xarray.core.dataset.Dataset

# Declarando variáveis de interesse

In [5]:
prec = ds.prec[:,0,0]
nf = ds.nf[:,0,0]
rh = ds.rh2m[:,0,0]
tmin = ds.tmin[:,0,0]
tmed = ds.tmed[:,0,0]
tmax = ds.tmax[:,0,0]

# Criando um array

In [6]:
array = np.column_stack((prec, nf, rh, tmin, tmed, tmax))

# Criando um DataFrame para dados diários

In [7]:
ds_diario = pd.DataFrame(array, index=ds.prec.time, columns=['Chuva', 'Focos', 'RH', 'Tmin', 'Tmed', 'Tmax'])
ds_diario.index.name = 'Dia' # Define o nome do index.

In [8]:
ds_diario

,Chuva,Focos,RH,Tmin,Tmed,Tmax
Dia,,,,,,
2021-01-01,4.105597,NaN,69.858345,23.483799,27.529976,34.520171
2021-01-02,1.155275,3.0,67.107567,24.123095,28.900791,35.697614
2021-01-03,6.212706,2.0,70.286880,24.610414,29.521808,35.477551
2021-01-04,5.626428,7.0,77.157661,23.557984,27.724243,32.264056
2021-01-05,8.518047,NaN,78.235023,22.473494,25.801426,33.195367
...,...,...,...,...,...,...
2021-12-27,0.413235,3.0,65.020134,24.234312,29.499307,36.670419
2021-12-28,0.134420,3.0,65.918297,24.663030,30.141633,36.074563
2021-12-29,0.609594,4.0,70.648399,25.676063,30.098496,34.261685


# Converte dados diários para mensal

In [9]:
# Converte o dados diário para mensal.
ds_mensal = ds_diario.resample('MS').agg({'Chuva':np.sum, 'Focos':np.sum, 'RH':np.mean, 'Tmin':np.mean, 'Tmed':np.mean, 'Tmax':np.mean})
ds_mensal.index.name = 'Mes' # Define o nome do index.

ds_mensal

,Chuva,Focos,RH,Tmin,Tmed,Tmax
Mes,,,,,,
2021-01-01,200.316615,32.0,76.424395,23.483123,27.079819,32.685140
2021-02-01,65.594583,29.0,70.851694,22.781502,27.364667,33.266521
2021-03-01,107.729518,39.0,74.877218,22.871949,27.044034,33.036237
2021-04-01,35.789609,51.0,67.900901,21.863085,26.309901,32.856372
2021-05-01,15.802381,54.0,64.572398,19.162125,24.333566,31.412830
2021-06-01,24.539930,85.0,66.192956,17.670453,22.803135,29.614364
2021-07-01,1.715199,394.0,42.366618,14.907985,22.177767,31.436564
2021-08-01,10.199708,711.0,40.737525,18.657894,25.653031,34.306103
2021-09-01,31.710627,2601.0,43.204789,23.868170,30.291995,38.131009


# Geração dos gráficos

In [10]:
import cufflinks as cf
from IPython.display import display, HTML

#making all charts public and setting a global theme
cf.set_config_file(sharing='public', theme='white', offline=True)

## Gráfico de linha

In [11]:
ds_diario[['Tmin', 'Tmed', 'Tmax']].iplot(kind='line', width=1)

In [12]:
ds_mensal[['Tmin', 'Tmed', 'Tmax']].iplot(kind='line', width=3)

In [13]:
ds_mensal['Tmed'].iplot(mode='lines+markers', size=4, colors="red", gridcolor="black", 
                        xTitle="Mês", yTitle="(°C)", title="Temperatura")

### Painel de figuras

In [14]:
ds_mensal[['Tmin', 'Tmed', 'Tmax']].iplot(y=["Tmin", "Tmed", "Tmax"],
                  width=2.0,
                  subplots=True,
                  colors=['blue','black','red'],
                  xTitle="Mês", yTitle="(°C)", title="Temperatura")

## Gráfico de barras vertical

In [15]:
ds_mensal['Chuva'].iplot(kind='bar')

In [16]:
ds_mensal[['Chuva', 'RH']].iplot(kind='bar',
                                 bargap=0.5,
                                 dimensions=(1200,600),
                                 xTitle="Mês", yTitle="Prec, RH", title="RH e Prec")

## Gráfico de barras horizontal

In [17]:
ds_mensal.iplot(kind="bar", y="Chuva",
                yTitle="Mês", xTitle="mm/mês", title="Titulo da figura",
                colors=["tomato"], bargap=0.5,
                sortbars=True,
                dimensions=(1000, 500),
                orientation="h")

## Gráfico de espalhamento

In [18]:
ds_diario.iplot(kind='scatter', x='RH', y='Tmed', mode='markers')

In [19]:
ds_diario.iplot(kind="scatter",
              x="RH", y='Tmed',
              mode='markers',
              colors="tomato", size=8, symbol="circle-open-dot",
              bestfit=True, bestfit_colors=["dodgerblue"],
              xTitle="(°C)", yTitle="(%)",
              title="RH x Tmed")

## Gráfico de boxplot

In [20]:
ds_diario.iplot(kind='box', boxpoints="outliers")

## Gráfico de áreas

In [21]:
ds_diario.Tmed.iplot(y="Tmed",
               fill=True,
               xTitle="Dia", yTitle="(°C)", title="Temperatura média diária - 2021",
               )

In [22]:
ds_diario[['Tmin', 'Tmed', 'Tmax']].iplot(
               keys=["Tmin", "Tmed", "Tmax",],
               subplots=True,
               colors=['blue', 'black', 'red'],
               fill=True,
               xTitle="Dia", yTitle="(°C)", title="Temperatura diária - 2021")

## Gráfico de histograma

In [23]:
ds_diario.Tmed.iplot(kind="hist",
                      bins=50, colors=["red"],
                      keys=["Tmed"],
                      dimensions=(600, 400),
                      title="Histograma de temperatura")

## Gráfico de bolhas

In [24]:
import plotly.express as px

In [25]:
ds_diario = ds_diario.reset_index()

In [26]:
Mes = pd.DatetimeIndex(ds_diario['Dia']).month  # Guarda apenas o mês (1, .., 12).

fig = px.scatter(ds_diario, x="Dia", y="Chuva", size='Chuva', color=Mes, 
                 title = 'Chuva diária')

fig.write_html("file.html")  # Salva no formato HTML.

fig.show()

### Painel de gráfico de bolhas

In [27]:
# Dado mensal desde 2019 a 2021.
ds = xr.open_dataset('./dados/chuva.2019.to.2021.nc')

In [28]:
prec = ds.PREC_surface[:,0,0]

In [29]:
df_prec = pd.DataFrame(prec, index=prec.time, columns=['Chuva'])
df_prec.index.name = 'Mes' # Define o nome do index.

In [30]:
df_prec = df_prec.reset_index()

In [40]:
import plotly.express as px

Mes = pd.DatetimeIndex(df_prec['Mes']).month  # Guarda apenas o mês (1, .., 12).
Ano = pd.DatetimeIndex(df_prec['Mes']).year  # Guarda apenas o ano.


Int64Index([2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019,
            2019, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020,
            2020, 2020, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021,
            2021, 2021, 2021],
           dtype='int64', name='Mes')

In [35]:
#help(px.scatter)

In [33]:
fig = px.scatter(df_prec, x='Mes', y='Chuva', size='Chuva', color=Mes, facet_row=Ano)
fig.show()